In [0]:
%sql
use catalog workspace;
use Schema labuser1;

select current_catalog(),current_schema()

In [0]:
spark.sql(f"LIST '/Volumes/workspace/labuser1/myfiles/'").display()

In [0]:
%sql
-- Create the bronze table first (if it doesn't exist)
-- Adjust schema as needed, here we let COPY INTO evolve schema
CREATE TABLE IF NOT EXISTS workspace.labuser1.Current_employees_bronze
USING DELTA;

-- Ingest CSVs and stamp source file info
COPY INTO workspace.labuser1.Current_employees_bronze
FROM (
  SELECT
    *,                               -- all CSV columns
    _metadata.file_name  AS src_file_name,
    _metadata.file_path  AS src_file_path,
    _metadata.file_modification_time AS src_file_mtime
  FROM csv.`/Volumes/workspace/labuser1/myfiles/`
)
FILEFORMAT = CSV
FORMAT_OPTIONS (
  'header' = 'true',
  'inferSchema' = 'true'
)
COPY_OPTIONS (
  'mergeSchema' = 'true'   -- optional but handy if columns change
);

In [0]:
%sql
drop table if exists current_customer_bronze;
drop table if exists workspace.labuser1.current_customer_bronze;
create table if not exists workspace.labuser1.current_customer_bronze(
  customer_id string,
  customer_name string,
  email string,
  country string,
  is_prime_member boolean
);

In [0]:
%sql
select * from current_customer_bronze;

In [0]:
spark.sql(f'''select * from CSV.`/Volumes/Workspace/labuser1/myfiles/`''').display()

In [0]:
#create the bronze raw ingestion table and include csv file name for rows
spark.sql(f"""
          Copy INTO workspace.labuser1.Current_customer_bronze
          from '/Volumes/workspace/labuser1/myfiles/'
          FILEFORMAT=CSV
          FORMAT_OPTIONS(
              'header'='true',
              'inferSchema'='true'
          )
          copy_options(
              'mergeSchema'='true'
          )
          """).display()

In [0]:
%sql
select * from workspace.labuser1.current_customer_bronze

In [0]:
%sql
-- Silver table built from Bronze
-- 1) Create (or replace) the silver table with cleaned, enriched data
CREATE OR REPLACE TABLE current_customer_silver AS
WITH cleaned AS (
  SELECT
      -- 1) Basic cleaning
      TRIM(customer_id)                        AS customer_id,
      -- Proper-case customer name (first letter of each word capitalized)
      INITCAP(REGEXP_REPLACE(TRIM(customer_name), '\s+', ' ')) AS customer_name,
      LOWER(TRIM(email))                       AS email,
      UPPER(TRIM(country))                     AS country,
      CAST(is_prime_member AS BOOLEAN)         AS is_prime_member,

      -- 2) Email validation & enrichment
      -- Simple email regex; you can tighten it as needed
      CASE 
        WHEN email RLIKE '^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
        THEN email
        ELSE NULL
      END                                       AS email_validated,
      -- domain after the '@' if valid
      CASE 
        WHEN email RLIKE '^[A-Za-z0-9._%+-]+@([A-Za-z0-9.-]+\.[A-Za-z]{2,})$'
        THEN REGEXP_EXTRACT(email, '@(.+)$', 1)
        ELSE NULL
      END                                       AS email_domain,

      -- 3) Country normalization
      -- Map common variants to a canonical country + ISO code
      CASE
        WHEN UPPER(TRIM(country)) IN ('US','USA','UNITED STATES','UNITED STATES OF AMERICA') THEN 'UNITED STATES'
        WHEN UPPER(TRIM(country)) IN ('UK','UNITED KINGDOM','GREAT BRITAIN','ENGLAND')       THEN 'UNITED KINGDOM'
        WHEN UPPER(TRIM(country)) IN ('UAE','UNITED ARAB EMIRATES')                           THEN 'UNITED ARAB EMIRATES'
        WHEN UPPER(TRIM(country)) IN ('INDIA','IN')                                           THEN 'INDIA'
        WHEN UPPER(TRIM(country)) IN ('AUSTRALIA','AU')                                       THEN 'AUSTRALIA'
        ELSE country
      END                                       AS country_canonical,
      CASE
        WHEN UPPER(TRIM(country)) IN ('US','USA','UNITED STATES','UNITED STATES OF AMERICA') THEN 'US'
        WHEN UPPER(TRIM(country)) IN ('UK','UNITED KINGDOM','GREAT BRITAIN','ENGLAND')       THEN 'GB'
        WHEN UPPER(TRIM(country)) IN ('UAE','UNITED ARAB EMIRATES')                           THEN 'AE'
        WHEN UPPER(TRIM(country)) IN ('INDIA','IN')                                           THEN 'IN'
        WHEN UPPER(TRIM(country)) IN ('AUSTRALIA','AU')                                       THEN 'AU'
        ELSE NULL
      END                                       AS country_code,

      -- 4) Audit columns
      current_timestamp()                       AS ingestion_ts,
      'bronze.current_customer_bronze'          AS record_source
  FROM current_customer_bronze
),
-- 5) Deduplicate by customer_id
--    Policy: keep the "best" record defined by:
--      a) validated email present,
--      b) prefer prime members,
--      c) latest ingestion_ts (same run gives same ts; replace with a real column if you have one)
scored AS (
  SELECT
      *,
      -- Prioritize validated email + prime members
      (CASE WHEN email_validated IS NOT NULL THEN 1 ELSE 0 END)  * 10
      + (CASE WHEN is_prime_member THEN 1 ELSE 0 END)            * 1 AS quality_score
  FROM cleaned
),
ranked AS (
  SELECT
      *,
      ROW_NUMBER() OVER (
        PARTITION BY customer_id
        ORDER BY quality_score DESC, ingestion_ts DESC
      ) AS rn
  FROM scored
)
SELECT
    customer_id,
    customer_name,
    email_validated AS email,        -- use the validated email
    email_domain,
    country_canonical AS country,
    country_code,
    is_prime_member,
    ingestion_ts,
    record_source
FROM ranked
WHERE rn = 1;

In [0]:
%sql
select * from current_customer_silver;

In [0]:
%sql
create or replace temp view temp_view_total_customers as
select
   country,
   count(*) as Total_count
from current_customer_silver
Group by country;

select * from temp_view_total_customers;
   

In [0]:
%sql
create table if not exists total_country_customers_gold(
  Country String,
  Total_count long
);

In [0]:
%sql
INSERT Overwrite Table total_country_customers_gold
select * from temp_view_total_customers;

In [0]:
%sql
select * from total_country_customers_gold;

In [0]:
%sql
describe history workspace.labuser1.total_country_customers_gold;